In [1]:
# from Tensorflow Neural Net Live Demo by Siraj Raval
from __future__ import print_function
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot = True)
# LEARNING POINT: what is one_hot <- a way of formatting data so that it is machine readable
# example [house, car, tooth, car] -> [0, 1, 2, 1] looks like tooth is > than car and house 
# instead can represent data as 'binary' such as house [1,0,0,0], car [0,1,0,0], tooth [0,0,1,0] but no one is greater than the other

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# hyperparameters
learning_rate = 0.001 # what we apply to weight process, lower is slower and more accurate, higher is faster but may over-fit skip
training_iters = 200000 # more iterations better but time consuming
# THINKING POINT: Automate training iterations?
batch_size = 128 # THINKING <- what is this exactly
display_step = 10

#network parameters
n_input = 784 # represents 28x 28 image
n_classes = 10 # there are ten possibilities (0-9)
dropout = 0.75 # prevents overfitting by randomly turning off paths to force new ways of learning for data

In [4]:
#placehold is gateway for data to flo into computational graph
x = tf.placeholder(tf.float32, [None,n_input]) # for image
y = tf.placeholder(tf.float32, [None,n_classes]) # for label
keep_prob = tf.placeholder(tf.float32) # for dropout

In [5]:
# convolutional layer take image, process, and tranform and becomes more abstract higher you go
# convolution is filter
# this first layer is a 2D layer
def conv2d(x, W, b, strides=1):
    # strides are our tensors
    x = tf.nn.conv2d(x, W, strides=[1,strides,strides,1], padding='SAME') # what is padding here?
    x = tf.nn.bias_add(x, b) # makes model more accurate
    return tf.nn.relu(x) # relu - rectified linear unit is activiation 

In [6]:
# samples of image ('pools') and take max learned combination of pixels in block 
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                         padding='SAME')

In [7]:
# create model
def conv_net(x, weights, biases, dropout):
    #reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1]) # format for computation graph to create
    # shape that we want is define as parameter with the pixel x pixel and our width and height reshaped to 1
    
    #convolutional layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    #Max pooling
    conv1 = maxpool2d(conv1, k=2)
    
    #next layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    # fully conected layer 
    # first reshape for conv layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # apply dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    #output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [8]:
# create weights
weights = {
    # 5x5 conv, with 1 input and 32 outputs
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    # 5x5 conv, 32 inputs and 64 outputs
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    # fully connected, 7x7x64 inputs and 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class predictions)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [9]:
# construct model
pred = conv_net(x, weights, biases, keep_prob)

# define loss and optimiser 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# initialize the variables
init = tf.global_variables_initializer()

# launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # keep training till amx iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # run optimiser op (back prop)
        sess.run(optimizer, feed_dict={x:batch_x, y:batch_y,
                                      keep_prob:dropout})
        if step % display_step == 0:
            # calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x:batch_x,
                                                             y:batch_y,
                                                             keep_prob:1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss = " + \
                  "{:.6f}".format(loss) + ", Training Accuracy = " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished")
    
    print("Testing Accuracy:", \
          sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 1280, Minibatch Loss = 18408.175781, Training Accuracy = 0.35938
Iter 2560, Minibatch Loss = 8727.071289, Training Accuracy = 0.51562
Iter 3840, Minibatch Loss = 5552.005859, Training Accuracy = 0.72656
Iter 5120, Minibatch Loss = 4999.621094, Training Accuracy = 0.75000
Iter 6400, Minibatch Loss = 2622.512451, Training Accuracy = 0.85156
Iter 7680, Minibatch Loss = 6228.501953, Training Accuracy = 0.75781
Iter 8960, Minibatch Loss = 3500.667725, Training Accuracy = 0.76562
Iter 10240, Minibatch Loss = 2238.226562, Training Accuracy = 0.83594
Iter 11520, Minibatch Loss = 891.497192, Training Accuracy = 0.94531
Iter 12800, Minibatch Loss = 2973.454834, Training Accuracy = 0.81250
Iter 14080, Minibatch Loss = 1167.507690, Training Accuracy = 0.90625
Iter 15360, Minibatch Loss = 1037.767090, Training Accuracy = 0.92969
Iter 16640, Minibatch Loss = 1780.015625, Training Accuracy = 0.89844
Iter 17920, Minibatch Loss = 1024.392090, Training Accuracy = 0.89844
Iter 19200, Minibatch Loss 